In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
# pip install openpyxl

In [3]:
nswmap = pd.read_excel('../data/nsw-site-location-and-details.xlsx')
print(nswmap.shape)
nswmap.head(3)

(2255, 12)


,Site,Site_distance,Community,Latitude,Longitude,Easting,Northing,Zone,Datum,Location name,Tenure,Date sampled
0,CYP_001,CYP_001_000,Cypress pine,-32.314901,145.820500,6424048,388969,55,MGA94,Bedooba State Conservation Area,National Park,2013-09-11
1,CYP_001,CYP_001_050,Cypress pine,-32.314476,145.820346,6424095,388954,55,MGA94,Bedooba State Conservation Area,National Park,2013-09-11
2,CYP_001,CYP_001_100,Cypress pine,-32.314051,145.820182,6424142,388938,55,MGA94,Bedooba State Conservation Area,National Park,2013-09-11


In [4]:
dung = pd.read_excel('../data/nsw-assessment-of-grazing-intensity.xlsx', sheet_name='animal_dung')
print(dung.shape)
dung.head(3)

(6019, 7)


,Community,Site,Site.1,Scale,Animal,Attribute,Counts
0,Black box,BLA_001_000,BLA_001,0.25,Kangaroo,dung,1
1,Black box,BLA_001_000,BLA_001,25.00,Cow,dung,28
2,Black box,BLA_001_000,BLA_001,25.00,Kangaroo,dung,10


In [5]:
dungmap = pd.merge(left=dung,
                   right=nswmap[['Community', 'Latitude', 'Longitude', 'Location name']], 
                   how='left',
                   on='Community')

dungmap.head()

MemoryError: Unable to allocate 173. MiB for an array with shape (5, 4527395) and data type object

In [ ]:
# dung1 = dung[dung['Attribute'].notna()]

# # keep only dung counts
# dung1 = dung[dung['Attribute'].str.contains('dung')]

# dung1.head()

In [ ]:
dungmap.info()

In [ ]:
dungmap['geometry'] = dungmap.apply(lambda x: Point((x.Longitude, 
                                                         x.Latitude)), 
                                        axis=1)
dungmap.head(3)

In [ ]:
dungmap = gpd.GeoDataFrame(dungmap, 
                           crs = 'EPSG:4326', 
                           geometry = dungmap['geometry'])

In [ ]:
dungmap.crs

In [ ]:
type(dungmap)

In [ ]:
dungmap.plot()

In [ ]:
geodata = gpd.read_file('../data/STE_2021_AUST_GDA2020.shp')
nsw = geodata[geodata['STE_NAME21']== 'New South Wales']
nsw.plot()

In [ ]:
nsw.crs = dungmap.crs

In [ ]:
nsw.crs

In [ ]:
nsw.plot()

In [ ]:
dungmap.head()

In [ ]:
type(nsw)

In [ ]:
nsw_map = gpd.sjoin(nsw, dungmap, predicate = 'within')

In [ ]:
border = nsw.loc[nsw['STE_NAME21'] == 'New South Wales']
border.shape

In [ ]:
ax = border.plot(figsize = (8, 10), color = 'lightgreen')
dungmap.plot( ax = ax, column = 'Community');
plt.show();

In [ ]:
border.geometry.centroid

In [ ]:
center = border.geometry.centroid[0]

In [ ]:
print(center)

## Folium requires a location point as an array with latitude first
but shapely Points have longitude first

In [ ]:
center.y

In [ ]:
# reverse the order when constructing the array for folium location
area_center = [center.y, center.x]

# check the order of area_center, the location for our folium map
print(area_center)

To create a folium map just call the folium Map() constructor
location is the only argument required by the constructor
Set area_center as the location of the folium map.
zoom_start is an optional argument
A setting of 12 should get us nice and close

In [ ]:
#create our map of NSW and show it
map_nsw = folium.Map(location =  area_center, zoom_start = 10)
map_nsw

#### Next, let's add our dung points


In [ ]:
dungmap.head(1)

In [ ]:
#draw our zip code area: 37207
map_nsw = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(border).add_to(map_nsw)

#iterate through stops_in_37207 to create locations and markers 
#for each bus stop
#remember for Folium locations, lat is listed first!!

for row_index, row_values in dungmap.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Location name'])
    icon=folium.Icon(color="blue",icon="poo", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, 
        icon = icon) 

    marker.add_to(map_nsw)
# map_37207.save('../maps/map37207.html')

#display our map
map_nsw